# Find proteins outside the 8p loss event (trans) that are commonly different between samples with and without the event

## Setup

In [1]:
import pandas as pd
import numpy as np
import os
import altair as alt

In [2]:
CHROMOSOME = "8"
ARM = "p"
TRANS_OR_CIS = "trans"

ttest_results_file = f"{CHROMOSOME}{ARM}_{TRANS_OR_CIS}effects.tsv"

ttest_results = pd.\
read_csv(ttest_results_file, sep="\t").\
rename(columns={"Name": "protein"}).\
set_index("protein")

In [3]:
ttest_results

,luad_Database_ID,lscc_Database_ID,brca_Database_ID,ovarian_Database_ID,brca_pvalue,colon_pvalue,hnscc_pvalue,lscc_pvalue,luad_pvalue,ovarian_pvalue,brca_diff,colon_diff,hnscc_diff,lscc_diff,luad_diff,ovarian_diff
protein,,,,,,,,,,,,,,,,
A1BG,NP_570602.2,NP_570602.2,NP_570602.2,NP_570602,0.982137,0.832246,0.885898,0.783533,0.259191,0.829152,-0.102606,-0.119042,-0.037599,0.312395,-0.442692,-0.294055
A2M,NP_000005.2|NP_001334353.1|NP_001334354.1|K4JD...,NP_000005.2|NP_001334353.1|NP_001334354.1,NP_000005.2,NP_000005,0.958650,0.869778,0.797487,0.855397,0.393935,0.942809,0.331652,-0.081407,-0.094712,0.301145,-0.461877,-0.244195
A2ML1,NaN,NP_653271.2|NP_001269353.1,NP_653271.2|NP_001269353.1,NaN,0.977601,NaN,0.995854,0.831661,NaN,NaN,-0.303609,NaN,0.005690,-1.153524,NaN,NaN
AAAS,NP_056480.1|NP_001166937.1,NP_056480.1|NP_001166937.1,NP_056480.1|NP_001166937.1,NP_056480,0.977241,0.672573,0.903656,0.950793,0.282024,0.995675,0.049927,-0.100606,0.020226,-0.037585,0.219718,0.030730
AACS,NP_076417.2|NP_001306769.1|NP_001306768.1,NP_076417.2|NP_001306769.1|NP_001306768.1,NP_076417.2|NP_001306769.1|NP_001306768.1,NP_076417,0.986910,0.701137,0.916413,0.756616,0.909540,0.968689,0.109607,0.089959,0.039000,-0.527445,-0.103594,0.132034
AAGAB,NP_078942.3|NP_001258814.1,NP_078942.3|NP_001258814.1,NP_078942.3|NP_001258815.1,NP_078942,0.860140,0.757541,0.925138,0.841119,0.738780,0.995675,0.367299,-0.078345,-0.026190,-0.174586,-0.119763,0.049414
AAK1,NaN,NP_055726.3,NP_055726.3,NP_055726,0.991094,0.906862,0.955778,0.910344,NaN,0.944506,0.020368,0.025289,-0.011070,0.081181,NaN,0.104969
AAMDC,NP_001303889.1|NP_001350493.1|NP_001303886.1|N...,NP_001303889.1|NP_001350493.1,NP_078960.1|NP_001303886.1|NP_001303887.1,NP_078960,0.992049,0.994357,0.881363,0.967873,0.953961,0.983804,-0.056287,-0.003264,0.052767,0.070351,-0.050954,0.126284
AAMP,NP_001289474.1|NP_001078.2,NP_001289474.1|NP_001078.2,NP_001289474.1|NP_001078.2,NP_001078,0.967734,0.670440,0.846881,0.756616,0.415096,0.968689,0.143079,-0.085926,0.048081,0.328295,-0.276061,0.091424


## Reshape the input dataframe
We want to get our table to have these columns:
- cancer_type
- protein
- Database_ID
- change
- p_value

Since some cancer types have database IDs and some don't, we'll slice out and reshape the info for each cancer type individually.

In [4]:
cancer_types = sorted(ttest_results.columns.to_series().str.split("_", n=1, expand=True)[0].unique())

long_results = pd.DataFrame()

for cancer_type in cancer_types:
    cancer_df = ttest_results.\
    loc[:, ttest_results.columns.str.startswith(cancer_type)].\
    dropna(axis="index", how="all").\
    reset_index(drop=False)
    
    # If the cancer type has database IDs, make a separate column that has them.
    # If not, create a column of NaNs (so that the tables all match)
    if f"{cancer_type}_Database_ID" in cancer_df.columns:
        cancer_df = cancer_df.rename(columns={f"{cancer_type}_Database_ID": "Database_ID"})
    else:
        cancer_df = cancer_df.assign(Database_ID=np.nan)
        
    # Rename the pvalue and diff columns to not have the cancer type
    cancer_df = cancer_df.rename(columns={
        f"{cancer_type}_pvalue": "adj_p",
        f"{cancer_type}_diff": "change"
    }).\
    assign(cancer_type=cancer_type)
    
    # Reorder the columns
    cancer_df = cancer_df[["cancer_type", "protein", "Database_ID", "adj_p", "change"]]
    
    # Append to the overall dataframe
    long_results = long_results.append(cancer_df)

# Drop duplicate rows and reset the index
long_results = long_results[~long_results.duplicated(keep=False)].\
reset_index(drop=True)

In [5]:
long_results

,cancer_type,protein,Database_ID,adj_p,change
0,brca,A1BG,NP_570602.2,0.982137,-0.102606
1,brca,A2M,NP_000005.2,0.958650,0.331652
2,brca,A2ML1,NP_653271.2|NP_001269353.1,0.977601,-0.303609
3,brca,AAAS,NP_056480.1|NP_001166937.1,0.977241,0.049927
4,brca,AACS,NP_076417.2|NP_001306769.1|NP_001306768.1,0.986910,0.109607
5,brca,AAGAB,NP_078942.3|NP_001258815.1,0.860140,0.367299
6,brca,AAK1,NP_055726.3,0.991094,0.020368
7,brca,AAMDC,NP_078960.1|NP_001303886.1|NP_001303887.1,0.992049,-0.056287
8,brca,AAMP,NP_001289474.1|NP_001078.2,0.967734,0.143079
9,brca,AAR2,NP_001258803.1,0.947234,0.167240


## Select the proteins with a significant change

In [10]:
prots = long_results[long_results["adj_p"] <= 0.05].reset_index(drop=True)
prots_cts = prots.groupby("cancer_type").count()[["protein"]]

fail_prots = long_results[long_results["adj_p"] > 0.05].reset_index(drop=True)
fail_cts = fail_prots.groupby("cancer_type").count()[["protein"]]

prots_cts.insert(0, "count_type", "Significant difference")
fail_cts.insert(0, "count_type", "No significant difference")

counts = prots_cts.append(fail_cts).sort_index().reset_index(drop=False)

alt.Chart(counts).mark_bar().encode(
    x=alt.X(
        "count_type",
        axis=alt.Axis(
            title=None,
            labels=False
        ),
        sort=["Significant difference"]
    ),
    y=alt.Y(
        "protein",
        axis=alt.Axis(
            title="Number of proteins"
        )
    ),
    color=alt.Color(
        "count_type",
        title=None,
        sort=["Significant difference"],
        scale=alt.Scale(
            domain=["Significant difference", "No significant difference"],
            range=["#2d3da4", "#d1d1d1"]
        )
    )
).facet(
    column=alt.Column(
        "cancer_type",
        title=None
    )
).properties(
    title="Chr 8p trans effects"
).configure_title(
    anchor="middle"
)

alt.FacetChart(...)

## Find how many cancers each protein was different in

In [7]:
prots_summary = prots.groupby("protein").agg(**{
    "cancers": ("cancer_type", lambda x: x.sort_values().drop_duplicates(keep="first").tolist())
})

prots_summary = prots_summary.\
assign(
    num_cancers=prots_summary["cancers"].apply(len),
    tmp_sort=prots_summary["cancers"].apply(lambda x: "".join(x))
).\
sort_values(by=["num_cancers", "tmp_sort"], ascending=[False, True]).\
drop(columns="tmp_sort")

prots_summary

,cancers,num_cancers
protein,,
CNOT8,"[brca, luad]",2
ATP6V1A,"[colon, luad]",2
ATP6V1E1,"[colon, luad]",2
ATP6V1H,"[colon, luad]",2
ATP6V1G1,[colon],1
CRELD2,[colon],1
DNAJC19,[colon],1
ERP29,[colon],1
NDUFAF2,[colon],1


In [8]:
prots_summary["num_cancers"].value_counts().sort_index(ascending=False)

2     4
1    56
Name: num_cancers, dtype: int64

## Save results

In [9]:
prots_summary = prots_summary.assign(
    cancers=prots_summary["cancers"].apply(lambda x: "_".join(x))
)

output_file = f"pancancer_summary_{CHROMOSOME}{ARM}_{TRANS_OR_CIS}.tsv"
prots_summary.to_csv(output_file, sep="\t")